In [128]:
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter("ignore")
import requests
from bs4 import BeautifulSoup

In [129]:

movie_tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
genres_top = pd.read_csv('top_genres_numbers.csv')
theatre_weekend = pd.read_csv('weekend_theaters_numbers.csv')


dataset = [movie_tmdb, movie_budget,genres_top, theatre_weekend]


In [130]:
conn = sqlite3.connect('zippedData/im.db/im.db')
cursor = conn.cursor()

# Get the main schema
cursor.execute('''
               SELECT name, sql 
               FROM sqlite_master 
               WHERE type='table';
               ''')
schema = cursor.fetchall()

# Print the schema
for name, schema in schema:
    print(f"Table: {name}")
    print(f"Schema: {schema}\n")

Table: movie_basics
Schema: CREATE TABLE "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
)

Table: directors
Schema: CREATE TABLE "directors" (
"movie_id" TEXT,
  "person_id" TEXT
)

Table: known_for
Schema: CREATE TABLE "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
)

Table: movie_akas
Schema: CREATE TABLE "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
)

Table: movie_ratings
Schema: CREATE TABLE "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
)

Table: persons
Schema: CREATE TABLE "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
)

Table: principals
Schema: CREATE TABLE "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" 

In [131]:
for i in dataset:
    display(i.head())
    print()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


,year,rank,genre,movies,gross,tickets,share
0,2023,1,Action,47,"$2,713,081,639","257,652,551",36.43%
1,2023,2,Adventure,24,"$1,455,186,660","138,194,353",19.54%
2,2023,3,Comedy,57,"$1,085,339,861","103,071,184",14.58%
3,2023,4,Drama,124,"$746,957,150","70,936,038",10.03%
4,2023,5,Horror,43,"$710,624,019","67,485,643",9.54%


,year,rank,movie,distributor,domestic_box_office,opening_weekend_box_office,max_theatre_count
0,2023,1,Barbie,Warner Bros.,635673355,162022044,4337
1,2023,2,The Super Mario Bros. Movie,Universal,574934330,146361865,4371
2,2023,3,Spider-Man: Across the Spider-Verse,Sony Pict…,381311319,120663589,4332
3,2023,4,Guardians of the Galaxy Vol 3,Walt Disney,358995815,118414021,4450
4,2023,5,Oppenheimer,Universal,324052905,82455420,3761


In [132]:
for i in dataset:
    i.info()
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   

In [133]:
for i in dataset:
    display(i.isna().sum())
    print()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

year       0
rank       0
genre      0
movies     0
gross      0
tickets    0
share      0
dtype: int64

year                            0
rank                            2
movie                           2
distributor                   133
domestic_box_office             0
opening_weekend_box_office    700
max_theatre_count              32
dtype: int64

In [134]:
for i in dataset:
    print(round((i.isna().sum().sort_values(ascending = False) / i.shape[0] * 100), 2))
    print()

vote_count           0.0
vote_average         0.0
title                0.0
release_date         0.0
popularity           0.0
original_title       0.0
original_language    0.0
id                   0.0
genre_ids            0.0
Unnamed: 0           0.0
dtype: float64

worldwide_gross      0.0
domestic_gross       0.0
production_budget    0.0
movie                0.0
release_date         0.0
id                   0.0
dtype: float64

share      0.0
tickets    0.0
gross      0.0
movies     0.0
genre      0.0
rank       0.0
year       0.0
dtype: float64

opening_weekend_box_office    9.45
distributor                   1.80
max_theatre_count             0.43
movie                         0.03
rank                          0.03
domestic_box_office           0.00
year                          0.00
dtype: float64



In [135]:
for i in dataset:
    display(i.duplicated().sum())
    print()

0

0

0

0

1. We have a dataset of 5 tables:
- movie_tmdb
- movie_budget
- genres_top
- data
- theatre_weekend
2. Our sql database consists of 8 tables:
- writers
- principals
- persons
- movie_ratings
- movie_akas
- directors
- movie_basics

You tube was founded in 2005, Netflix in 200. Streaming platforms changed the industry of cinema. We include movies from 2010 only

Importing neccessary information from database to dataframe

In [136]:
        
sql_query = pd.read_sql_query ('''
                               SELECT
                               mb.original_title,
                               mb.start_year,
                               mb.runtime_minutes,
                               mb.genres,
                               mr.averagerating,
                               mr.numvotes,
                               p.primary_name,
                               p.primary_profession
                               FROM movie_akas
                               JOIN movie_ratings mr
                               USING(movie_id)
                               JOIN movie_basics mb
                               USING(movie_id)
                               JOIN writers
                               USING(movie_id)
                               JOIN persons p
                               USING(person_id)
                               WHERE start_year >= 2010
                               AND region = 'US'
                               GROUP BY primary_title
                               ''', conn)

df = pd.DataFrame(sql_query, columns = [
    'original_title', 'start_year', 'runtime_minutes', 
    'genres', 'averagerating', 'numvotes',
    'primary_name', 'primary_profession']
)

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22534 entries, 0 to 22533
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   original_title      22534 non-null  object 
 1   start_year          22534 non-null  int64  
 2   runtime_minutes     20935 non-null  float64
 3   genres              22470 non-null  object 
 4   averagerating       22534 non-null  float64
 5   numvotes            22534 non-null  int64  
 6   primary_name        22534 non-null  object 
 7   primary_profession  22527 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.4+ MB


In [138]:
df.head(10)

,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,primary_name,primary_profession
0,#1 Serial Killer,2013,87.0,Horror,5.6,40,Koji Steven Sakai,"producer,writer,director"
1,#5,2013,68.0,"Biography,Comedy,Fantasy",6.8,6,Ricky Bardy,"producer,director,editor"
2,#Captured,2017,81.0,Thriller,2.8,212,Lizze Gordon,"actress,producer,writer"
3,#DigitalLivesMatter,2016,NaN,Comedy,6.2,19,Cas Sigers-Beedles,"producer,writer,director"
4,#Disneyland60,2015,45.0,Documentary,8.8,5,M.L. Martin,"writer,producer,cinematographer"
5,#Enough,2015,NaN,Drama,2.4,7,Damion Stevenson,"actor,writer,miscellaneous"
6,#FollowFriday,2016,90.0,Thriller,2.8,186,Micho Rutare,"miscellaneous,writer,director"
7,#Horror,2015,101.0,"Crime,Drama,Horror",3.0,3092,Tara Subkoff,"actress,producer,director"
8,#Lucky Number,2015,80.0,Comedy,5.1,425,David Cornue,"producer,writer,composer"
9,#O2LForever,2015,95.0,"Biography,Documentary,Music",9.2,66,Michael Goldfine,"producer,director,writer"


In [139]:
df.original_title = df.original_title.str.lower()
df.genres = df.genres.str.lower()

In [140]:
df.runtime_minutes = df.runtime_minutes.fillna(df.runtime_minutes.mean())


In [141]:
df = df.rename(columns={'original_title': 'movie',
                        'start_year':'release_year'})

In [142]:
df = df.dropna(subset=['genres', 'primary_profession'])


In [143]:
df.release_year.sort_values(ascending=True)

11264    2010
15792    2010
15790    2010
4115     2010
15776    2010
         ... 
5400     2019
21740    2019
13828    2019
5443     2019
19364    2019
Name: release_year, Length: 22463, dtype: int64

Exploring table genre to define top 10 genres by revenue during 2010-2023

In [144]:
genres_top.genre = genres_top.genre.str.lower()

Converting "gross" and "ticket" to float type

In [145]:
genres_top.gross = genres_top.gross.str.replace(r'[\$,]', '').str.strip().astype(float)
genres_top.tickets = genres_top.tickets.str.replace(r'[\$,]', '').str.strip().astype(float)

Top 10 genres during 2010-2023 period of time

In [146]:
top_10 = genres_top.groupby('genre')['gross'].sum().sort_values(ascending=False).head(10)
total_sum = top_10.sum()
percentage_values = (top_10 / total_sum) * 100
print(round(percentage_values,2))

genre
action               28.26
adventure            27.42
drama                12.90
comedy               10.49
thriller/suspense     8.20
horror                6.25
musical               2.32
romantic comedy       2.29
black comedy          1.09
documentary           0.77
Name: gross, dtype: float64


Preprocessing table movie_budget

In [147]:
movie_budget.release_date = pd.to_datetime(movie_budget.release_date)

In [148]:
movie_budget.movie = movie_budget.movie.str.lower()

In [149]:
movie_budget = movie_budget[movie_budget.release_date >= '2010-01-01']

In [150]:
movie_budget.production_budget = movie_budget.production_budget.str.replace(r'[\$,]', '').str.strip().astype(float)
movie_budget.domestic_gross= movie_budget.domestic_gross.str.replace(r'[\$,]', '').str.strip().astype(float)

In [151]:
movie_budget = movie_budget.drop(columns=['id',
                                          'worldwide_gross',
                                          'release_date'])

In [152]:
movie_budget.movie = movie_budget.movie.str.lower()

Preprocessing table theatre_weekend

In [153]:
theatre_weekend = theatre_weekend.drop(columns=[
    'year', 'rank'])

In [154]:
theatre_weekend.movie = theatre_weekend.movie.str.lower()

In [155]:
theatre_weekend = theatre_weekend.dropna(subset=['domestic_box_office',
                               'opening_weekend_box_office',
                               'movie', 
                               ])

In [156]:
theatre_weekend.distributor = theatre_weekend.distributor.fillna("No Distributor")

Preprocessing table movie_tmdb

In [157]:
movie_tmdb.release_date = pd.to_datetime(movie_tmdb.release_date)

In [158]:
movie_tmdb = movie_tmdb[movie_tmdb.release_date >= '2010-01-01']

In [159]:
movie_tmdb = movie_tmdb.drop(columns=[
    'Unnamed: 0', 'genre_ids', 'id', 'original_language',
    'original_title'
])

In [160]:
movie_tmdb.title = movie_tmdb.title.str.lower()
movie_tmdb =movie_tmdb.rename(columns={'title':'movie'})

In [161]:
# data = data.drop(columns=[
#     'movie_odid', 'production_budget', 'domestic_box_office',
#     'international_box_office'
# ])

In [162]:
# data.movie_name = data.movie_name.str.lower()
# data = data.rename(columns={'movie_name':'movie'})

Creating a master table.  
Merging df(imported from database) with:
- movie_budget
- theatre_weekend
- movie_tmbd
- data

In [163]:
movie_budget.movie = movie_budget.movie.str.replace(r'[/\,.:;<>|""!?@&*()]', '').str.strip()
df.movie = df.movie.str.replace(r'[/\,.:;<>|""!?@&*()]', '').str.strip()
movie_tmdb.movie = movie_tmdb.movie.str.replace(r'[/\,.:;<>|""!?@&*()]', '').str.strip()
theatre_weekend.movie = theatre_weekend.movie.str.replace(r'[/\,.:;<>|""!?@&*()]', '').str.strip()

In [164]:
dataset_new = [df, movie_tmdb, movie_budget,genres_top, theatre_weekend]
for i in dataset_new:
    i.info()
    print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22463 entries, 0 to 22533
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie               22463 non-null  object 
 1   release_year        22463 non-null  int64  
 2   runtime_minutes     22463 non-null  float64
 3   genres              22463 non-null  object 
 4   averagerating       22463 non-null  float64
 5   numvotes            22463 non-null  int64  
 6   primary_name        22463 non-null  object 
 7   primary_profession  22463 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.5+ MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26291 entries, 0 to 26516
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   popularity    26291 non-null  float64       
 1   release_date  26291 non-null  datetime64[ns]
 2   movie         26291 non-nu

In [165]:
# !pip install fuzzywuzzy
# from fuzzywuzzy import fuzz


# def find_similar_values(col1, col2):
#     for name1 in movie_budget[col1]:
#         for name2 in theatre_weekend[col2]:
#             if fuzz.partial_ratio(name1, name2) > 80:
#                 print(f"Similar values found: {name1} and {name2}")

# # Using the function to find similar values
# find_similar_values('movie', 'movie')

In [166]:
# merge_1 = pd.merge(movie_tmdb, df, how='left', on="movie")
# merge_2 = pd.merge(merge_1,theatre_weekend, how='left', on="movie")
# data1 = pd.merge(merge_2,movie_budget, how='left', on="movie")



In [167]:
merge_1 = pd.merge(movie_tmdb, df, on="movie")
merge_2 = pd.merge(merge_1,theatre_weekend, on="movie")
data = pd.merge(merge_2,movie_budget, on="movie")

In [168]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1648 entries, 0 to 1647
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   popularity                  1648 non-null   float64       
 1   release_date                1648 non-null   datetime64[ns]
 2   movie                       1648 non-null   object        
 3   vote_average                1648 non-null   float64       
 4   vote_count                  1648 non-null   int64         
 5   release_year                1648 non-null   int64         
 6   runtime_minutes             1648 non-null   float64       
 7   genres                      1648 non-null   object        
 8   averagerating               1648 non-null   float64       
 9   numvotes                    1648 non-null   int64         
 10  primary_name                1648 non-null   object        
 11  primary_profession          1648 non-null   object      

In [169]:
data.head()

,popularity,release_date,movie,vote_average,vote_count,release_year,runtime_minutes,genres,averagerating,numvotes,primary_name,primary_profession,distributor,domestic_box_office,opening_weekend_box_office,max_theatre_count,production_budget,domestic_gross
0,28.734,2010-03-26,how to train your dragon,7.7,7610,2010,98.0,"action,adventure,animation",8.1,611299,William Davies,"writer,producer,miscellaneous",Paramount…,217581232,43732319,4060,165000000.0,217581232.0
1,28.515,2010-05-07,iron man 2,6.8,12368,2010,124.0,"action,adventure,sci-fi",7.0,657690,Jack Kirby,"writer,animation_department,miscellaneous",Paramount…,312433331,128122480,4390,170000000.0,312433331.0
2,27.920,2010-07-16,inception,8.3,22186,2010,148.0,"action,adventure,sci-fi",8.8,1841066,Christopher Nolan,"writer,producer,director",Warner Bros.,292576195,62785337,3792,160000000.0,292576195.0
3,26.691,2010-02-11,percy jackson the olympians the lightning thief,6.1,4229,2010,118.0,"adventure,family,fantasy",5.9,165108,Craig Titley,"producer,writer,miscellaneous",20th Cent…,88768303,31236067,3396,95000000.0,88768303.0
4,24.445,2010-06-17,toy story 3,7.7,8340,2010,103.0,"adventure,animation,comedy",8.3,682218,Andrew Stanton,"writer,actor,producer",Walt Disney,415004880,110307189,4028,200000000.0,415004880.0


We are dropping dublicates,reordering columns, correct their names, convert datatypes to the appropriate types.

In [170]:
data = data.drop(columns=['release_year', 'domestic_gross'])

In [171]:
data  = data.rename(columns={'averagerating':'average_rating', 'numvotes':'num_votes'})

In [172]:
data.duplicated().sum()

133

In [173]:
data = data.drop_duplicates()

In [174]:
data.runtime_minutes = data.runtime_minutes.astype('int64')
data.domestic_box_office = data.domestic_box_office.astype('int64')
data.opening_weekend_box_office = data.opening_weekend_box_office.astype('int64')
data.max_theatre_count = data.max_theatre_count.astype('int64')
data.production_budget = data.production_budget.astype('int64')


In [175]:

data['rio_percentage'] = (data['domestic_box_office'] / data['production_budget']) * 100
data['rio_percentage'] = data['rio_percentage'].astype('int64')
data['multiplier'] = round((data['domestic_box_office'] / data['opening_weekend_box_office']), 1)
data['popularity'] = round((data['popularity']), 1)

In [176]:
data.head()

,popularity,release_date,movie,vote_average,vote_count,runtime_minutes,genres,average_rating,num_votes,primary_name,primary_profession,distributor,domestic_box_office,opening_weekend_box_office,max_theatre_count,production_budget,rio_percentage,multiplier
0,28.7,2010-03-26,how to train your dragon,7.7,7610,98,"action,adventure,animation",8.1,611299,William Davies,"writer,producer,miscellaneous",Paramount…,217581232,43732319,4060,165000000,131,5.0
1,28.5,2010-05-07,iron man 2,6.8,12368,124,"action,adventure,sci-fi",7.0,657690,Jack Kirby,"writer,animation_department,miscellaneous",Paramount…,312433331,128122480,4390,170000000,183,2.4
2,27.9,2010-07-16,inception,8.3,22186,148,"action,adventure,sci-fi",8.8,1841066,Christopher Nolan,"writer,producer,director",Warner Bros.,292576195,62785337,3792,160000000,182,4.7
3,26.7,2010-02-11,percy jackson the olympians the lightning thief,6.1,4229,118,"adventure,family,fantasy",5.9,165108,Craig Titley,"producer,writer,miscellaneous",20th Cent…,88768303,31236067,3396,95000000,93,2.8
4,24.4,2010-06-17,toy story 3,7.7,8340,103,"adventure,animation,comedy",8.3,682218,Andrew Stanton,"writer,actor,producer",Walt Disney,415004880,110307189,4028,200000000,207,3.8


In [177]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 0 to 1647
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   popularity                  1515 non-null   float64       
 1   release_date                1515 non-null   datetime64[ns]
 2   movie                       1515 non-null   object        
 3   vote_average                1515 non-null   float64       
 4   vote_count                  1515 non-null   int64         
 5   runtime_minutes             1515 non-null   int64         
 6   genres                      1515 non-null   object        
 7   average_rating              1515 non-null   float64       
 8   num_votes                   1515 non-null   int64         
 9   primary_name                1515 non-null   object        
 10  primary_profession          1515 non-null   object        
 11  distributor                 1515 non-null   object      

In [181]:
master_data = data[['release_date', 'movie', 'genres', 'vote_count', 
             'num_votes', 'vote_average','popularity', 
             'average_rating', 'domestic_box_office', 
             'production_budget', 'rio_percentage', 
             'opening_weekend_box_office', 'multiplier',
             'max_theatre_count', 'distributor', 
             'primary_name', 'primary_profession']]

In [183]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 0 to 1647
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   release_date                1515 non-null   datetime64[ns]
 1   movie                       1515 non-null   object        
 2   genres                      1515 non-null   object        
 3   vote_count                  1515 non-null   int64         
 4   num_votes                   1515 non-null   int64         
 5   vote_average                1515 non-null   float64       
 6   popularity                  1515 non-null   float64       
 7   average_rating              1515 non-null   float64       
 8   domestic_box_office         1515 non-null   int64         
 9   production_budget           1515 non-null   int64         
 10  rio_percentage              1515 non-null   int64         
 11  opening_weekend_box_office  1515 non-null   int64       

In [190]:
master_data.to_csv('master_data.csv', index=False)

In [194]:
master_data = pd.read_csv('master_data.csv')
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   release_date                1515 non-null   object 
 1   movie                       1515 non-null   object 
 2   genres                      1515 non-null   object 
 3   vote_count                  1515 non-null   int64  
 4   num_votes                   1515 non-null   int64  
 5   vote_average                1515 non-null   float64
 6   popularity                  1515 non-null   float64
 7   average_rating              1515 non-null   float64
 8   domestic_box_office         1515 non-null   int64  
 9   production_budget           1515 non-null   int64  
 10  rio_percentage              1515 non-null   int64  
 11  opening_weekend_box_office  1515 non-null   int64  
 12  multiplier                  1515 non-null   float64
 13  max_theatre_count           1515 

In [192]:
master_dedup = pd.read_csv('master_dedup.csv')

In [193]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   movie_name                   1290 non-null   object 
 1   genres                       1290 non-null   object 
 2   percentage_ROI_gross_budget  1290 non-null   int64  
 3   gross                        1290 non-null   int64  
 4   budget                       1290 non-null   int64  
 5   average_rating               1290 non-null   float64
 6   number_vote                  1290 non-null   float64
 7   primary_name                 1290 non-null   object 
 8   primary_profession           1290 non-null   object 
 9   release_date                 1290 non-null   object 
 10  release_year                 1290 non-null   int64  
 11  runtime_minutes              1290 non-null   float64
 12  distributor                  1290 non-null   object 
 13  max_theatre_count 

In [195]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   release_date                1515 non-null   object 
 1   movie                       1515 non-null   object 
 2   genres                      1515 non-null   object 
 3   vote_count                  1515 non-null   int64  
 4   num_votes                   1515 non-null   int64  
 5   vote_average                1515 non-null   float64
 6   popularity                  1515 non-null   float64
 7   average_rating              1515 non-null   float64
 8   domestic_box_office         1515 non-null   int64  
 9   production_budget           1515 non-null   int64  
 10  rio_percentage              1515 non-null   int64  
 11  opening_weekend_box_office  1515 non-null   int64  
 12  multiplier                  1515 non-null   float64
 13  max_theatre_count           1515 